In [1]:
import cPickle
import sklearn 
import numpy as np
import scipy
import csv

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape, Merge
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta
from keras.constraints import unitnorm, maxnorm
from keras.regularizers import l2
from keras.layers.advanced_activations import PReLU

from sklearn.metrics import roc_auc_score



Using Theano backend.


In [2]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    return scipy.sparse.csr_matrix(( loader['data'], loader['indices'], loader['indptr']),
                     shape = loader['shape'])
        
def load_csv(filename):
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter = '\n')
        array = [float(row[0]) for row in reader]
        return array
    
def load_feature_names(filename):
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter= '\n')
        array = [row for row in reader]
        return array

In [3]:
path = "simplified_data/"
data_train = load_sparse_csr(path+'data_train.npz')
data_test = load_sparse_csr(path+'data_test.npz')
label_train = load_csv(path+'label_train.csv')
label_test = load_csv(path+'label_test.csv')
features_names = load_feature_names('data/feature_names.csv')

In [4]:
label_train = np.array(label_train)

In [5]:
#import theano
#data_train = theano.sparse.csr_matrix("data_train.npz")
#data_test = theano.sparse.csr_matrix('data_test.npz')
#from sknn.mlp import Classifier, Layer
#model =Classifier(layers=[Layer("Softmax")], learning_rate = 0.001, n_iter=25, dropout_rate = 0.9)
#model.fit(data_train.toarray(), label_train)

In [9]:
data_train.shape

(18750, 10000)

In [8]:
from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
fselect = SelectKBest(chi2 , k=10000)
data_train = fselect.fit_transform(data_train, label_train)
data_test = fselect.transform(data_test)

In [10]:
data_traind = data_train.todense()

In [14]:
from sknn.mlp import Classifier, Layer

nn = Classifier(
    layers=[
        Layer("Tanh", units=50),
        Layer("Softmax")],
        learning_rate=0.001,
        n_iter=25,
        dropout_rate=0.,
        verbose = True,
        batch_size = 10)
nn.fit(data_train.toarray(), label_train)

Classifier(batch_size=10, callback=None, debug=False, dropout_rate=0.0,
      f_stable=0.001,
      hidden0=<sknn.nn.Layer `Tanh`: units=50, name=u'hidden0', frozen=False>,
      layers=[<sknn.nn.Layer `Tanh`: units=50, name=u'hidden0', frozen=False>, <sknn.nn.Layer `Softmax`: units=2, name=u'output', frozen=False>],
      learning_momentum=0.9, learning_rate=0.001, learning_rule=u'sgd',
      loss_type=None, n_iter=25, n_stable=10,
      output=<sknn.nn.Layer `Softmax`: units=2, name=u'output', frozen=False>,
      random_state=None, regularize=None, valid_set=None, valid_size=0.0,
      verbose=True, warning=None, weight_decay=None, weights=None)

In [15]:
def score (a, b):
    return 1- np.absolute(a - b).mean()
predicted_label = nn.predict(data_test.todense())
print("Neural network - Score on test_data : ", score(predicted_label, label_test))
print("Neural network - Score on train_data : ", score(nn.predict(data_train.todense()), label_train))

('Neural network - Score on test_data : ', 0.49992299520000005)
('Neural network - Score on train_data : ', 0.50003404231111115)


In [16]:

scores = []
alphas = np.logspace(-7,0,30)
for alpha in alphas:
    nn = Classifier(
    layers=[
        Layer("Tanh", units=50),
        Layer("Softmax")],
        learning_rate=0.0001,
        n_iter=25,
        dropout_rate=0.,
        verbose = True,
        batch_size = 16)
    nn.fit(data_train.toarray(), label_train)
    predicted_label = nn.predict(data_test.todense())
    scores.append(score(predicted_label, label_test))
    

In [18]:
maxi = scores[0]
amaxi = 0
for i, score in enumerate(scores):
    if maxi< score:
        maxi = score
        amaxi = i
print amaxi, maxi

18 0.4996780544


In [19]:
print data_train.todense().shape
label_train.shape

(18750, 20000)


(18750,)

In [23]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, input_dim=20, init='uniform'))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(64, init='uniform'))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(2, init='uniform'))
model.add(Activation('softmax'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

model.fit(data_train.todense(), label_train, nb_epoch=20, batch_size=16)
score = model.evaluate(data_test.todense(), label_test, batch_size=16)


Epoch 1/20


ValueError: Shape mismatch: x has 20000 cols (and 16 rows) but y has 20 rows (and 64 cols)
Apply node that caused the error: Dot22(<TensorType(float32, matrix)>, <TensorType(float32, matrix)>)
Inputs types: [TensorType(float32, matrix), TensorType(float32, matrix)]
Inputs shapes: [(16, 20000), (20, 64)]
Inputs strides: [(80000, 4), (256, 4)]
Inputs values: ['not shown', 'not shown']

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.